<a href="https://colab.research.google.com/github/ttr314/clog-loss-stall-catchers-benchmark/blob/master/StClair_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


The Presentation topic will be:
 
Using the most applicable software tools for this project, please complete the following tasks and be prepared to present on your steps and findings: 

1.   Open the MobData.txt file attached and import to prepare and clean the data. Once you have finished preparing the data, save your file as mob####.CSV. Explain how you prepared or cleaned the data. 
2. Create a dashboard using the CSV file you created in part 1.  The dashboard should show a summary with different charts using the columns below. The dashboard should clearly display what are the most common types of vehicles, when is traffic the heaviest, and what direction are vehicles travelling.  What steps did you use to create the dashboard? 
*   Timestamp
*   vehicleType 
*    entrance  
*   exit
*    Qty 
3. Create an algorithm that can be used to predict traffic volumes based on the data provided in part 1. 










---



---



**Preview text file**

In [ ]:
# Preview text file
with open('Mobdata.txt') as f:
    lines = f.read(150)
print(lines)

{"intersectionId":1,"intersectionDescription":"Dorchester Road and Huron Church Road","traffic":[{"timeStamp":"2021-06-27T00:00:00","vehicleType":"Art


*   Preview of file shows data is likely stored in JSON format.
*   Will import data as JSON into python dataframe

**Import file as JSON and preview**

In [ ]:
# Set up python workspace including json package 
import pandas as pd
import json  

In [ ]:
# Import text file
df = pd.read_json('Mobdata.txt')

In [ ]:
# Preview imported file
df.head(3)

,intersectionId,intersectionDescription,traffic
0,1,Dorchester Road and Huron Church Road,"{'timeStamp': '2021-06-27T00:00:00', 'vehicleT..."
1,1,Dorchester Road and Huron Church Road,"{'timeStamp': '2021-06-27T00:00:00', 'vehicleT..."
2,1,Dorchester Road and Huron Church Road,"{'timeStamp': '2021-06-27T00:00:00', 'vehicleT..."




*   After initial import as JSON, we observe last column *traffic* is nested, containing another layer of columns.
*   Apply method to further unravel the data nested within *traffic*





**Additional steps with JSON import to unravel data in nested columns**

In [ ]:
with open('Mobdata.txt','r') as f:
    data = json.loads(f.read())
df = pd.json_normalize(data, \
                            meta =['intersectionId','intersectionDescription'],\
                            record_path =['traffic'])

In [ ]:
#preview file
df.head(3)

,timeStamp,vehicleType,entrance,exit,qty,isDaylightSavingsTime,intersectionId,intersectionDescription
0,2021-06-27T00:00:00,ArticulatedTruck,N,S,3,True,1,Dorchester Road and Huron Church Road
1,2021-06-27T00:00:00,Light,N,S,4,True,1,Dorchester Road and Huron Church Road
2,2021-06-27T00:00:00,Light,S,N,1,True,1,Dorchester Road and Huron Church Road




*   Import complete with one data element per column for each row



**Initital review of columns (Pre-clean analysis)**

In [ ]:
# Distinct values by column
for col in ['intersectionId','intersectionDescription','vehicleType','entrance','exit','isDaylightSavingsTime']:
  print(df[col].value_counts(),'\n\n')

1    6514
Name: intersectionId, dtype: int64 


Dorchester Road and Huron Church Road    6514
Name: intersectionDescription, dtype: int64 


Light               4633
ArticulatedTruck    1731
SingleUnitTruck       98
Bicycle               34
Bus                   18
Name: vehicleType, dtype: int64 


S    2937
N    2250
E     857
W     470
Name: entrance, dtype: int64 


S    2538
N    2457
E     923
W     596
Name: exit, dtype: int64 


True    6514
Name: isDaylightSavingsTime, dtype: int64 




*   Investigate column data types for further cleaning



In [ ]:
df.dtypes

timeStamp                  object
vehicleType                object
entrance                   object
exit                       object
qty                         int64
isDaylightSavingsTime        bool
intersectionId             object
intersectionDescription    object
dtype: object

**Convert applicable columns to categorical** 

In [ ]:
for col in ["intersectionId","intersectionDescription", "vehicleType","entrance","exit" ]:
  df[col] = df[col].astype('category')

**Convert "timeStamp" column to a pandas date-time data type**

In [ ]:
df['timeStamp'] = pd.to_datetime(df['timeStamp']) 

**Confirm succesful conversion of columns to analytical data types**



In [ ]:
df.dtypes

timeStamp                  datetime64[ns]
vehicleType                      category
entrance                         category
exit                             category
qty                                 int64
isDaylightSavingsTime                bool
intersectionId                   category
intersectionDescription          category
dtype: object

**Export file to CSV, mobdata.csv**

In [ ]:
df.to_csv('mobdata.csv', index=False)



---



---



---



# **EXPT, PREMODELING**

In [ ]:
import plotly.express as px
import numpy as np

grouped = df.groupby('timeStamp')
df2 = grouped['qty'].agg(np.sum)


In [ ]:
# Aggregate data by minute
df2 = df.groupby('timeStamp', as_index=False)[['qty']].sum()

In [ ]:
df2.head(3)

,timeStamp,qty
0,2021-06-27 00:00:00,8
1,2021-06-27 00:01:00,4
2,2021-06-27 00:02:00,7


In [ ]:
fig = px.line(df2, x='timeStamp', y="qty")
fig.show()